In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'seaborn'

In [6]:
def get_model_performance(X, y, model_func, epochs=100):
    full_history =  []
    confusion_matrices= []
    for i in range(len(X)):
        X_test, y_test = X[i], y[i]
        X_train, y_train = [ X[j] for j in range(len(X)) if j != i], [y[j] for j in range(len(X)) if j != i]
        X_train, y_train = np.vstack(X_train), np.hstack(y_train)
        one_hot_train_labels = to_categorical(y_train)
        one_hot_test_labels = to_categorical(y_test)
        model = model_func(X_train)
        history = model.fit(X_train, one_hot_train_labels, epochs= epochs, batch_size=64, validation_data = (X_test, one_hot_test_labels))
        full_history.append(history)
        pred = model.predict(X_test)
        pred = np.argmax(pred, axis=1)
        confusion_matrix = metrics.confusion_matrix(pred, y_test)
        confusion_matrices.append(confusion_matrix)

    return full_history, confusion_matrices

In [3]:
def FDDLM_Model(data):
    model = keras.Sequential(name="FDDLM_Sequential")
    print(data.shape[1:])
    model.add( layers.Input(shape=data.shape[1:]) )
    model.add( layers.Dense(300, activation='relu', name="Dense_1"))
    model.add(layers.Flatten())
    model.add( layers.Dense(100, activation='relu', name="Dense_2"))
    model.add( layers.Dense(5, activation='softmax', name='output'))
    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model

In [4]:
X1 = np.load("../COPD_dataset/hold-out-validation/cp_data.npy")
X2 = np.load("../COPD_dataset/hold-out-validation/md1_data.npy")
X3 = np.load("../COPD_dataset/hold-out-validation/md2_data.npy")
X4 = np.load("../COPD_dataset/hold-out-validation/vb_data.npy")

y = [X1[:,-1],X2[:,-1],X3[:,-1],X4[:,-1],]

X = [X1[:,:-1], X2[:,:-1], X3[:,:-1], X4[:,:-1]]

In [7]:
fddlm_history, fddlm_confusion_matrices = get_model_performance(model_func= FDDLM_Model, X= X, y= y, epochs=100)

(144,)
Model: "FDDLM_Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense_1 (Dense)             (None, 300)               43500     
                                                                 
 flatten (Flatten)           (None, 300)               0         
                                                                 
 Dense_2 (Dense)             (None, 100)               30100     
                                                                 
 output (Dense)              (None, 5)                 505       
                                                                 
Total params: 74,105
Trainable params: 74,105
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
52/52 [==============================] - 2s 6ms/step - loss: 1.3428 - accuracy: 0.4377 - val_loss: 1.4116 - val_accuracy: 0.3926
Epoch 2/100
52/52 [======

In [8]:
fddlm_500_history, fddlm_500_confusion_matrices = get_model_performance(model_func= FDDLM_Model, X= X, y= y, epochs=500)

(144,)
Model: "FDDLM_Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense_1 (Dense)             (None, 300)               43500     
                                                                 
 flatten_4 (Flatten)         (None, 300)               0         
                                                                 
 Dense_2 (Dense)             (None, 100)               30100     
                                                                 
 output (Dense)              (None, 5)                 505       
                                                                 
Total params: 74,105
Trainable params: 74,105
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
52/52 [==============================] - 1s 6ms/step - loss: 1.3514 - accuracy: 0.4392 - val_loss: 1.2187 - val_accuracy: 0.4982
Epoch 2/500
52/52 [======

In [9]:
def save_history(filename, history):
    np_history = []
    for i in range(len(history)):
        his = history[i]
        np_history.append(his.history)
    np.save(filename, np_history)

In [10]:
save_history('fddlm_history',fddlm_history)
save_history('fddlm_500_history',fddlm_500_history)
np.save("fddlm_cfs",fddlm_confusion_matrices)
np.save("fddlm_500_cfs",fddlm_500_confusion_matrices)